# وظیفه طبقه‌بندی متن

همان‌طور که اشاره کردیم، ما بر روی یک وظیفه ساده طبقه‌بندی متن بر اساس مجموعه داده **AG_NEWS** تمرکز خواهیم کرد. این وظیفه شامل طبقه‌بندی تیترهای خبری به یکی از ۴ دسته زیر است: جهان، ورزش، تجارت و علم/فناوری.

## مجموعه داده

این مجموعه داده در ماژول [`torchtext`](https://github.com/pytorch/text) گنجانده شده است، بنابراین می‌توانیم به راحتی به آن دسترسی پیدا کنیم.


In [1]:
import torch
import torchtext
import os
import collections
os.makedirs('./data',exist_ok=True)
train_dataset, test_dataset = torchtext.datasets.AG_NEWS(root='./data')
classes = ['World', 'Sports', 'Business', 'Sci/Tech']

اینجا، `train_dataset` و `test_dataset` شامل مجموعه‌هایی هستند که جفت‌هایی از برچسب (شماره کلاس) و متن را به ترتیب بازمی‌گردانند، برای مثال:


In [2]:
list(train_dataset)[0]

(3,
 "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.")

خب، بیایید اولین ۱۰ تیتر جدید از مجموعه داده‌های خود را چاپ کنیم:


In [5]:
for i,x in zip(range(5),train_dataset):
    print(f"**{classes[x[0]]}** -> {x[1]}")


**Sci/Tech** -> Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again.
**Sci/Tech** -> Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\which has a reputation for making well-timed and occasionally\controversial plays in the defense industry, has quietly placed\its bets on another part of the market.
**Sci/Tech** -> Oil and Economy Cloud Stocks' Outlook (Reuters) Reuters - Soaring crude prices plus worries\about the economy and the outlook for earnings are expected to\hang over the stock market next week during the depth of the\summer doldrums.
**Sci/Tech** -> Iraq Halts Oil Exports from Main Southern Pipeline (Reuters) Reuters - Authorities have halted oil export\flows from the main pipeline in southern Iraq after\intelligence showed a rebel militia could strike\infrastructure, an oil official said on Saturday.
**Sci/Tech** -> Oil prices soar to

چون مجموعه داده‌ها تکرارگر هستند، اگر بخواهیم داده‌ها را چندین بار استفاده کنیم باید آن را به لیست تبدیل کنیم:


In [3]:
train_dataset, test_dataset = torchtext.datasets.AG_NEWS(root='./data')
train_dataset = list(train_dataset)
test_dataset = list(test_dataset)

## توکنیزاسیون

حالا باید متن را به **اعداد** تبدیل کنیم که بتوانند به صورت تنسورها نمایش داده شوند. اگر بخواهیم نمایشی در سطح کلمات داشته باشیم، باید دو کار انجام دهیم:
* استفاده از **توکنایزر** برای تقسیم متن به **توکن‌ها**
* ساخت یک **واژگان** از این توکن‌ها.


In [4]:
tokenizer = torchtext.data.utils.get_tokenizer('basic_english')
tokenizer('He said: hello')

['he', 'said', 'hello']

In [5]:
counter = collections.Counter()
for (label, line) in train_dataset:
    counter.update(tokenizer(line))
vocab = torchtext.vocab.vocab(counter, min_freq=1)

با استفاده از واژگان، می‌توانیم رشته توکن‌شده خود را به راحتی به مجموعه‌ای از اعداد رمزگذاری کنیم:


In [19]:
vocab_size = len(vocab)
print(f"Vocab size if {vocab_size}")

stoi = vocab.get_stoi() # dict to convert tokens to indices

def encode(x):
    return [stoi[s] for s in tokenizer(x)]

encode('I love to play with my words')

Vocab size if 95810


[599, 3279, 97, 1220, 329, 225, 7368]

## نمایش متن به روش Bag of Words

از آنجا که کلمات معنا را منتقل می‌کنند، گاهی می‌توانیم با نگاه کردن به کلمات جداگانه، بدون توجه به ترتیب آن‌ها در جمله، معنای یک متن را درک کنیم. برای مثال، هنگام دسته‌بندی اخبار، کلماتی مانند *هواشناسی*، *برف* احتمالاً به *پیش‌بینی هوا* اشاره دارند، در حالی که کلماتی مانند *سهام*، *دلار* به احتمال زیاد به *اخبار مالی* مربوط می‌شوند.

نمایش برداری **Bag of Words** (BoW) یکی از رایج‌ترین روش‌های سنتی نمایش برداری است. در این روش، هر کلمه به یک شاخص بردار مرتبط می‌شود و عنصر بردار تعداد دفعات وقوع یک کلمه در یک سند مشخص را نشان می‌دهد.

![تصویری که نشان می‌دهد چگونه نمایش برداری Bag of Words در حافظه نمایش داده می‌شود.](../../../../../lessons/5-NLP/13-TextRep/images/bag-of-words-example.png)

> **Note**: می‌توانید به BoW به عنوان مجموع تمام بردارهای یک‌داغ (one-hot-encoded) برای کلمات جداگانه در متن نیز فکر کنید.

در زیر مثالی از نحوه تولید نمایش Bag of Words با استفاده از کتابخانه Scikit Learn در پایتون آورده شده است:


In [7]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[1, 1, 0, 2, 0, 0, 0, 0, 0]], dtype=int64)

برای محاسبه بردار کیسه کلمات از نمایش برداری مجموعه داده AG_NEWS، می‌توانیم از تابع زیر استفاده کنیم:


In [20]:
vocab_size = len(vocab)

def to_bow(text,bow_vocab_size=vocab_size):
    res = torch.zeros(bow_vocab_size,dtype=torch.float32)
    for i in encode(text):
        if i<bow_vocab_size:
            res[i] += 1
    return res

print(to_bow(train_dataset[0][1]))

tensor([2., 1., 2.,  ..., 0., 0., 0.])


> **توجه:** در اینجا از متغیر جهانی `vocab_size` برای مشخص کردن اندازه پیش‌فرض واژگان استفاده می‌کنیم. از آنجا که اغلب اندازه واژگان بسیار بزرگ است، می‌توانیم اندازه واژگان را به پرتکرارترین کلمات محدود کنیم. سعی کنید مقدار `vocab_size` را کاهش دهید و کد زیر را اجرا کنید و ببینید چگونه بر دقت تأثیر می‌گذارد. انتظار می‌رود کمی کاهش دقت داشته باشید، اما نه به صورت چشمگیر، در عوض عملکرد بالاتر.


## آموزش طبقه‌بند BoW

حالا که یاد گرفتیم چگونه نمایه کیسه کلمات (Bag-of-Words) را برای متن خود بسازیم، بیایید یک طبقه‌بند را بر اساس آن آموزش دهیم. ابتدا باید داده‌های خود را برای آموزش به گونه‌ای تبدیل کنیم که تمام نمایش‌های برداری موقعیتی به نمایه کیسه کلمات تبدیل شوند. این کار با استفاده از تابع `bowify` به‌عنوان پارامتر `collate_fn` در `DataLoader` استاندارد کتابخانه torch امکان‌پذیر است:


In [21]:
from torch.utils.data import DataLoader
import numpy as np 

# this collate function gets list of batch_size tuples, and needs to 
# return a pair of label-feature tensors for the whole minibatch
def bowify(b):
    return (
            torch.LongTensor([t[0]-1 for t in b]),
            torch.stack([to_bow(t[1]) for t in b])
    )

train_loader = DataLoader(train_dataset, batch_size=16, collate_fn=bowify, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, collate_fn=bowify, shuffle=True)

حالا بیایید یک شبکه عصبی طبقه‌بندی ساده تعریف کنیم که شامل یک لایه خطی است. اندازه بردار ورودی برابر با `vocab_size` است و اندازه خروجی مربوط به تعداد کلاس‌ها (۴) می‌باشد. چون ما در حال حل یک وظیفه طبقه‌بندی هستیم، تابع فعال‌سازی نهایی `LogSoftmax()` است.


In [22]:
net = torch.nn.Sequential(torch.nn.Linear(vocab_size,4),torch.nn.LogSoftmax(dim=1))

اکنون حلقه آموزش استاندارد PyTorch را تعریف می‌کنیم. از آنجا که مجموعه داده ما بسیار بزرگ است، برای هدف آموزشی خود فقط برای یک دوره آموزش خواهیم داد و گاهی حتی کمتر از یک دوره (مشخص کردن پارامتر `epoch_size` به ما امکان محدود کردن آموزش را می‌دهد). همچنین دقت تجمعی آموزش را در طول آموزش گزارش خواهیم داد؛ فرکانس گزارش‌دهی با استفاده از پارامتر `report_freq` مشخص می‌شود.


In [24]:
def train_epoch(net,dataloader,lr=0.01,optimizer=None,loss_fn = torch.nn.NLLLoss(),epoch_size=None, report_freq=200):
    optimizer = optimizer or torch.optim.Adam(net.parameters(),lr=lr)
    net.train()
    total_loss,acc,count,i = 0,0,0,0
    for labels,features in dataloader:
        optimizer.zero_grad()
        out = net(features)
        loss = loss_fn(out,labels) #cross_entropy(out,labels)
        loss.backward()
        optimizer.step()
        total_loss+=loss
        _,predicted = torch.max(out,1)
        acc+=(predicted==labels).sum()
        count+=len(labels)
        i+=1
        if i%report_freq==0:
            print(f"{count}: acc={acc.item()/count}")
        if epoch_size and count>epoch_size:
            break
    return total_loss.item()/count, acc.item()/count

In [25]:
train_epoch(net,train_loader,epoch_size=15000)

3200: acc=0.8028125
6400: acc=0.8371875
9600: acc=0.8534375
12800: acc=0.85765625


(0.026090790722161722, 0.8620069296375267)

## بی‌گرام‌ها، تری‌گرام‌ها و اِن‌گرام‌ها

یکی از محدودیت‌های روش کیسه کلمات این است که برخی کلمات بخشی از عبارات چندکلمه‌ای هستند. برای مثال، کلمه «هات‌داگ» معنای کاملاً متفاوتی نسبت به کلمات «هات» و «داگ» در سایر زمینه‌ها دارد. اگر همیشه کلمات «هات» و «داگ» را با همان بردارها نمایش دهیم، ممکن است مدل ما را دچار سردرگمی کند.

برای رفع این مشکل، از **نمایش اِن‌گرام‌ها** در روش‌های طبقه‌بندی اسناد استفاده می‌شود. در این روش، فراوانی هر کلمه، دوکلمه‌ای یا سه‌کلمه‌ای به‌عنوان ویژگی مفیدی برای آموزش طبقه‌بندها در نظر گرفته می‌شود. برای مثال، در نمایش بی‌گرام، علاوه بر کلمات اصلی، تمام جفت کلمات نیز به واژگان اضافه می‌شوند.

در زیر مثالی از نحوه تولید نمایش کیسه کلمات بی‌گرام با استفاده از Scikit Learn آورده شده است:


In [26]:
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', min_df=1)
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
bigram_vectorizer.fit_transform(corpus)
print("Vocabulary:\n",bigram_vectorizer.vocabulary_)
bigram_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()


Vocabulary:
 {'i': 7, 'like': 11, 'hot': 4, 'dogs': 2, 'i like': 8, 'like hot': 12, 'hot dogs': 5, 'the': 16, 'dog': 0, 'ran': 14, 'fast': 3, 'the dog': 17, 'dog ran': 1, 'ran fast': 15, 'its': 9, 'outside': 13, 'its hot': 10, 'hot outside': 6}


array([[1, 0, 1, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int64)

عیب اصلی روش N-gram این است که اندازه واژگان به‌سرعت بسیار زیادی رشد می‌کند. در عمل، لازم است که نمایش N-gram را با برخی تکنیک‌های کاهش ابعاد، مانند *embeddingها*، ترکیب کنیم که در واحد بعدی درباره آن‌ها صحبت خواهیم کرد.

برای استفاده از نمایش N-gram در مجموعه داده **AG News**، باید یک واژگان خاص ngram بسازیم:


In [27]:
counter = collections.Counter()
for (label, line) in train_dataset:
    l = tokenizer(line)
    counter.update(torchtext.data.utils.ngrams_iterator(l,ngrams=2))
    
bi_vocab = torchtext.vocab.vocab(counter, min_freq=1)

print("Bigram vocabulary length = ",len(bi_vocab))

Bigram vocabulary length =  1308842


ما می‌توانیم از همان کدی که در بالا استفاده کردیم برای آموزش دسته‌بند استفاده کنیم، اما این روش از نظر حافظه بسیار ناکارآمد خواهد بود. در واحد بعدی، دسته‌بند بی‌گرام را با استفاده از تعبیه‌ها آموزش خواهیم داد.

> **توجه:** شما فقط می‌توانید آن ان‌گرام‌هایی را نگه دارید که بیش از تعداد مشخصی در متن ظاهر می‌شوند. این کار باعث می‌شود بی‌گرام‌های کم‌تکرار حذف شوند و به طور قابل توجهی ابعاد کاهش یابد. برای انجام این کار، پارامتر `min_freq` را به مقدار بالاتری تنظیم کنید و تغییر طول واژگان را مشاهده کنید.


## فراوانی اصطلاح و فراوانی معکوس سند (TF-IDF)

در نمایش BoW، وقوع کلمات به طور مساوی وزن‌دهی می‌شود، بدون توجه به خود کلمه. با این حال، واضح است که کلمات پرتکرار مانند *a*، *in* و غیره برای دسته‌بندی اهمیت کمتری نسبت به اصطلاحات تخصصی دارند. در واقع، در بیشتر وظایف پردازش زبان طبیعی (NLP)، برخی کلمات نسبت به دیگران مرتبط‌تر هستند.

**TF-IDF** مخفف **فراوانی اصطلاح–فراوانی معکوس سند** است. این یک تغییر در روش کیسه کلمات (BoW) است که به جای مقدار دودویی 0/1 که نشان‌دهنده حضور یک کلمه در یک سند است، از یک مقدار اعشاری استفاده می‌شود که به فراوانی وقوع کلمه در مجموعه داده مرتبط است.

به طور رسمی‌تر، وزن $w_{ij}$ یک کلمه $i$ در سند $j$ به صورت زیر تعریف می‌شود:
$$
w_{ij} = tf_{ij}\times\log({N\over df_i})
$$
که در آن:
* $tf_{ij}$ تعداد وقوع‌های $i$ در $j$ است، یعنی همان مقدار BoW که قبلاً دیده‌ایم
* $N$ تعداد اسناد در مجموعه داده است
* $df_i$ تعداد اسنادی است که کلمه $i$ را در کل مجموعه داده شامل می‌شود

مقدار TF-IDF یعنی $w_{ij}$ به طور متناسب با تعداد دفعاتی که یک کلمه در یک سند ظاهر می‌شود افزایش می‌یابد و با تعداد اسنادی که در مجموعه داده شامل آن کلمه هستند تنظیم می‌شود. این امر کمک می‌کند تا برای این واقعیت که برخی کلمات بیشتر از دیگران ظاهر می‌شوند، جبران شود. به عنوان مثال، اگر یک کلمه در *هر* سند مجموعه داده ظاهر شود، $df_i=N$ و $w_{ij}=0$ خواهد بود، و این اصطلاحات کاملاً نادیده گرفته می‌شوند.

شما می‌توانید به راحتی بردارسازی TF-IDF متن را با استفاده از Scikit Learn ایجاد کنید:


In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2))
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[0.43381609, 0.        , 0.43381609, 0.        , 0.65985664,
        0.43381609, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ]])

## نتیجه‌گیری

با این حال، اگرچه نمایش‌های TF-IDF وزن فرکانسی به کلمات مختلف اختصاص می‌دهند، اما قادر به نمایش معنا یا ترتیب نیستند. همان‌طور که زبان‌شناس مشهور جی. آر. فرث در سال ۱۹۳۵ گفت: «معنای کامل یک کلمه همیشه وابسته به متن است و هیچ مطالعه‌ای درباره معنا بدون در نظر گرفتن متن نمی‌تواند جدی گرفته شود.» در ادامه دوره یاد خواهیم گرفت که چگونه اطلاعات متنی را با استفاده از مدل‌سازی زبان استخراج کنیم.



---

**سلب مسئولیت**:  
این سند با استفاده از سرویس ترجمه هوش مصنوعی [Co-op Translator](https://github.com/Azure/co-op-translator) ترجمه شده است. در حالی که ما تلاش می‌کنیم دقت را حفظ کنیم، لطفاً توجه داشته باشید که ترجمه‌های خودکار ممکن است شامل خطاها یا نادرستی‌ها باشند. سند اصلی به زبان اصلی آن باید به عنوان منبع معتبر در نظر گرفته شود. برای اطلاعات حساس، توصیه می‌شود از ترجمه حرفه‌ای انسانی استفاده کنید. ما مسئولیتی در قبال سوءتفاهم‌ها یا تفسیرهای نادرست ناشی از استفاده از این ترجمه نداریم.
